In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer


from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white, het_breuschpagan, het_goldfeldquandt, linear_reset

In [ ]:
df = pd.read_csv('transactions-2023-01-11.csv')

In [ ]:
def drop_excess_columns(data):
    # Drop high cardinality columns
    data = data.drop(columns=["Transaction Number", "Property ID", "Transaction Size (sq.m)", "Parking", "Project"])
    # Drop low-cardinality columns
    data = data.drop(columns=["Registration type", "Master Project"])
    # Drop leaky columns
    data = data.drop(columns=["Transaction sub type", "Property Type", "Room(s)", "No. of Buyer", "No. of Seller", 'Nearest Mall', 'Nearest Landmark'])
    return data


In [ ]:
df = drop_excess_columns(df)

In [ ]:
# def get_oil_price():
#     # data from https://tradingeconomics.com/commodity/crude-oil
#     r = requests.get('https://markets.tradingeconomics.com/chart?s=cl1:com&interval=1d&span=5y&securify=new&url=/commodity/crude-oil&AUTH=j6Bt4pj3JEXdfgKlsPWa%2BhKpyYuaZeaGdJeVvofKg%2F50KkIli0%2Fe4jDtQXrDXz5r&ohlc=0')
    
#     oil_data = pd.DataFrame(r.json()['series'][0]['data']).rename(columns={"y": "Oil Price"})
#     oil_data = oil_data[(oil_data['date'] > '2021-03-01') & (oil_data['date'] < '2022-02-04')]

#     return oil_data[['date', 'Oil Price']]
    

In [ ]:
# oil_data = get_oil_price()
# oil_data.to_csv (r'oil_data.csv')
# oil_data

In [ ]:
oil_data = pd.read_csv('oil_data.csv', index_col=0)
oil_data.head(2)

In [ ]:
def drop_period_after_war(data):
    war_date = '2022-02-24'
    return data[data['Transaction Date'] < war_date]

In [ ]:
df = drop_period_after_war(df)

In [ ]:
def merge_oil_to_data(data, oil):
    data['date_without_time'] = pd.to_datetime(data['Transaction Date']).dt.strftime('%Y-%m-%d')
    oil['date_without_time'] = pd.to_datetime(oil['date']).dt.strftime('%Y-%m-%d')
    data = data.merge(oil, left_on='date_without_time', right_on='date_without_time')
    return data

In [ ]:
df = merge_oil_to_data(df, oil_data)

In [ ]:
df.head(3)

In [ ]:
def clean_outliers_in_data(data):
    quantiles = data.quantile(0.98)
    data = data[(data['Amount'] < quantiles['Amount']) & (data['Property Size (sq.m)'] < quantiles['Property Size (sq.m)'])]
    data = data[data['Property Sub Type'].isin(["Commercial", "Flat", "Hotel Apartment", "Hotel Rooms",  "Office", "Residential", "Residential / Attached Villas","Residential Flats", "Stacked Townhouses", "Villa"])]
    data = data.dropna()
    return data

In [ ]:
cleaned_df = clean_outliers_in_data(df)

In [ ]:
cleaned_df.head(3)

In [ ]:
from pandas.plotting import scatter_matrix

def plot_matrix(data):
    scatter_matrix(data[['Amount', 'Property Size (sq.m)']], figsize=(12, 8))
    scatter_matrix(data[['Amount', 'Oil Price']], figsize=(12, 8))
    return data['Property Size (sq.m)'].describe()

In [ ]:
plot_matrix(cleaned_df)

In [ ]:
def preparing_data_before_training(data, drop, renameColumns):
    data = data.drop(drop, axis=1)
    data = data.rename(columns=renameColumns)
    return data

In [ ]:
prepared_data = preparing_data_before_training(
    cleaned_df, 
    ['date_without_time', 'Transaction Date', 'date'],
    {
        'Property Size (sq.m)': 'Property_Size',
        'Property Sub Type': 'Property_Sub_Type',
        'Nearest Metro': 'Nearest_Metro',
        'Nearest Mall': 'Nearest_Mall',
        'Nearest Landmark': 'Nearest_Landmark',
        'Oil Price': 'Oil_Price',
        'Transaction Type': 'Transaction_Type',
        "Is Free Hold?": 'Is_Free_Hold'
    }
)

In [ ]:
import statsmodels.formula.api as smf #  + C(Nearest_Mall) + C(Nearest_Landmark) 
formula = 'Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + C(Nearest_Metro)+ Oil_Price + C(Is_Free_Hold)'
sm_data = sm.add_constant(prepared_data)
model = smf.ols(formula=formula, data=sm_data)
results = model.fit()


In [ ]:
results.summary()

In [ ]:
fig = sm.graphics.plot_regress_exog(results, "Property_Size")
fig.tight_layout(pad=1.0);

In [ ]:
pred_ols = results.get_prediction()

fig, ax = plt.subplots(figsize=(8, 6))

x = prepared_data['Property_Size']
y = prepared_data['Amount']

ax.plot(x, y, "o", label="data")
ax.plot(x, results.fittedvalues, "o", label="OLS")

ax.legend(loc="best");

$\DeclareMathOperator{\Corr}{Corr}$
# Multicollinearity

###### def:Multicollinearity is a linear relationship between explanatory variables. =>  $Corr(X_i, X_j) > 0$
###### When we face multicollinearity:
1. When regressors mean relatively the same (ex: height of a building and number of floors)
2. Natural relationships between variables (ex: year of work and age)

###### Consequences of multicollinearity:
1. Estimates __remain unbiased__
2. Increased standard errors
3. Wide confidence intervals
4. Insignificant coefficients
5. Estimates become sensitive to slight changes

###### How to detect multicollinearity:
1. small changes lead to serious changes in estimates
2. insignificant estimates due to big standard errors
3. unexpected signs of estimates

###### Indicators of multicollinearity:
1. Strong covariance between regressors - calculate sample Var-Cov matrix
2. Large value of VIF (Variance inflation factor) <br>
$ VIF(x_j) = \frac{1}{1 - R_j^2}$
,where $ R_j^2$ is determination coefficient for regression <br>
In VIF method, we pick each feature and regress it against all of the other features. Generally, a VIF above 5 indicates a high multicollinearity. <br>
https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

###### Dealing with multicollinearity: (ЖЕЛАТЕЛЬНО ДОПОЛНИТЬ)
1. Increase sample size
2. Change functional form
3. Impose some priory restrictions on parameters
4. Dimension reduction / Principal component analysis (PCA)
5. Ridge and LASSO regressions

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import patsy

def VIF(data, formula):  # from explanatory matrix and formula
    """
    Calculates VIF coefficients for given dataframe. Returns DataFrame.
    
    Arguments:
    data - DataFrame that have been used in regression
    formula - Formula used in regression as a string, example: 'Amount ~ C(Transaction_Type) + C(Area)'
    
    Interpretation:
    The variance inflation factor is a measure for the increase of the variance of the parameter estimates 
    if an additional variable, given by exog_idx is added to the linear regression. 
    It is a measure for multicollinearity of the design matrix, exog.

    One recommendation is that if VIF is greater than 5, 
    then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables,
    and the parameter estimates will have large standard errors because of this.
        
    Sources:
    https://en.wikipedia.org/wiki/Variance_inflation_factor
    https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
    
    """
    f = formula  
    y, X = patsy.dmatrices(f, data, return_type='matrix')

    vif_data = pd.DataFrame()
    vif_data["feature"] = X.design_info.column_names

    vif_values = []
    for i in range(len(X.design_info.column_names)):
        vif_values.append(variance_inflation_factor(X, i))

    vif_data['VIF'] = vif_values
    return vif_data


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF2(model, results):  # from model
    """
    Calculates VIF coefficients for given model. Returns DataFrame.
    
    Arguments:
    model - fitted regression model from Statsmodels
    
    Interpretation:
    The variance inflation factor is a measure for the increase of the variance of the parameter estimates 
    if an additional variable, given by exog_idx is added to the linear regression. 
    It is a measure for multicollinearity of the design matrix, exog.

    One recommendation is that if VIF is greater than 5, 
    then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables,
    and the parameter estimates will have large standard errors because of this.
        
    Sources:
    https://en.wikipedia.org/wiki/Variance_inflation_factor
    https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html
    
    """
    vif_data = pd.DataFrame()
    vif_data["feature"] = model.exog_names

    vif_values = []
    for i in range(len(model.exog_names)):
        vif_values.append(variance_inflation_factor(results.model.exog, i))

    vif_data['VIF'] = vif_values
    return vif_data

In [ ]:
VIF = VIF(sm_data, 'Amount ~ C(Transaction_Type) + C(Usage) + C(Area) + C(Property_Sub_Type) + Property_Size + Oil_Price + C(Is_Free_Hold)')  # sm_data, 'Amount ~ C(Transaction_Type) + C(Area) + C(Nearest_Landmark) + Oil_Price + C(Is_Free_Hold)'
VIF.head(3)

In [ ]:
VIF.sort_values('VIF', ascending=False).head(10)

In [ ]:
y_sub = pd.DataFrame(model.exog, columns=model.exog_names)['C(Property_Sub_Type)[T.Flat]']
X_sub = pd.DataFrame(model.exog, columns=model.exog_names).drop(columns='C(Property_Sub_Type)[T.Flat]')

In [ ]:
model_sub = sm.OLS(y_sub, X_sub)
results_sub = model_sub.fit()

In [ ]:
results_sub.summary()

In [ ]:
pd.DataFrame(model.exog, columns=model.exog_names).head(3)

$\DeclareMathOperator{\Corr}{Corr}$
# Heteroskedasticity

###### def: The disturbances are homoscedastic if the variance of $\epsilon_i$ is a constant $\sigma ^{2}$; otherwise, they are heteroscedastic. =>  $ Var(\epsilon_i) \neq \sigma^{2} $
###### When we face heteroskedasticity:
The most basic heteroskedastic example is household consumption. The variance in consumption increases with an increase in income—directly proportional. Because when the income is low, the variance in consumption is also low. Low-income people spend predominantly on necessary items and bills—less variance. In contrast, with the increase in income, people tend to buy luxurious items and develop a plethora of habits—less predictable.

###### Consequences of heteroskedasticity:
1. Standard errors (SE) of $\hat{\beta}$ calculated incorrectly => t-tests and F-tests are wrong
2. OLS estimates are not BLUE anymore => efficiency loss
3. Estimates are unbiased and consistent: $\mathbb{E}(\hat{\beta}) = \beta$,  ${\hat{\beta} \to \beta}$ as probability of estimates converge to probability of parameter

###### How to detect heteroskedasticity:
1. White test
2. Goldfeld-Quandt test
3. Glejser test
4. Breusch-Pagan test

### White test

H0: Homoskedasticity <br>
H1: Heteroskedasticity <br>

Step 1. Estimate the regression <br>
example: $ y_i = \beta_0 + \beta_1 x_{1i} + \beta_2 x_{2i} + \epsilon_i $

Step 2. Save residuals $ \hat{\epsilon_i} $ <br>
Step 3. Build the auxiliary model <br>
Regress squares of saved $ \hat{\epsilon_i} $ __on__ regressors $x_{1i}$ and $x_{2i}$, their squares and their product <br>
$ \hat{\epsilon_i}^2 = \alpha_0 + \alpha_1 x_{1i} + \alpha_2 x_{2i} + \alpha_3 x_{1i}^2 + \alpha_4 x_{2i}^2 + \alpha_5 x_{1i} x_{2i} + \epsilon_i $ <br>
Step 4. Calculate $R^2$ for auxiliary model <br>
Step 5. Calculate test statistic <br>
$ W = n R^2 \sim \chi_{k-1}^2 $, where k is the number of estimated coefficients in auxiliary regression <br>

#### Interpretation
High R^2 means that we have the dependance between the variance of the disturbance term and the regressors

#### Suitable for large samples and doesn't imply the normality of residuals




In [ ]:
# Долго считается, не дождался
def white_test(residuals, exog):
    #perform White's test
    white_test = het_white(residuals,  exog)
    #define labels to use for output of White's test
    labels = ['Test Statistic', 'Test Statistic p-value', 'F-Statistic', 'F-Test p-value']
    #print results of White's test
    output = dict(zip(labels, white_test))
    return output

### Goldfeld-Quandt test

H0: Homoskedasticity $ \sigma_i^2 = \sigma^2$,  $ \forall_1i = 1, ..., n $<br>
H1: Heteroskedasticity  $ \sigma_i^2 \sim x_{ji}$ for some $ x_j $, variance is proportional to some regressor $ x_j $<br>

Step 1. Sort data by variable $ x_j $, that we suspect bring heteroskedasticity

Step 2. Split sample into 3 parts.

Step 3. Estimate regressions on the subsamples for first $ n_1 $ and last $ n_2 $ observations and calculate $ RSS_1$ and $ RSS_2$


Step 4. Calculate test statistic: <br>
$ F = \frac{\frac{RSS_2}{n_2 - k}}{\frac{RSS_1}{n_1 - k}}
$

Step 5. Under H0 <br>
$ F \sim F_{n2-k, n1-k} $


#### Suitable for small samples and doesn't imply the normality of residuals

In [ ]:
def goldfeldquandt_test(results, model): # from model
    """
    Calculates F-Stat and p-value for given fitted model. Returns Dictionary.
    
    Arguments:
    results - fitted regression model from Statsmodels
    model - unfitted model from Statsmodel
    idx - index of a column as a int, by which explanatory variables are sorted
    
    """
    labels = ['F-Statistic', 'F-Test p-value', 'Ordering used']
    values = []
    for i in range(len(model.exog_names)):
        values.append(het_goldfeldquandt(results.model.endog, results.model.exog, idx=i))

    output = pd.DataFrame(values, columns = labels)
    output['feature'] = model.exog_names
    
    return output


In [ ]:
#het = goldfeldquandt_test(results, model)
#het

### Glejser test

H0: Homoskedasticity $ \sigma_i^2 = \sigma^2$,  $ \forall_1i = 1, ..., n $<br>
H1: Heteroskedasticity  $ \sigma_i^2 \sim x_{j}^\gamma $ for $ \gamma = 1, 1/2 $ or $ -1 $ <br>
Glejser test assumes that we can have the dependance between the $ \sigma_i^2 $ and some powers of $ x_{j} $, so the $ \sigma_i^2 $ is proportional to $ x_{j}^\gamma $, where $ \gamma $ can be 1, 1/2 or -1


Step 1. Estimate regression on the whole sample <br>

Step 2. Predict residuals $ \epsilon_i $  <br>

Step 3. Take absolute values of predicted $ \epsilon_i $ and regress the folowing: <br>
$ |\epsilon_i| = \alpha + \beta x_i + \mu_i $, for  i = 1, ..., n <br>
$ |\epsilon_i| = \alpha + \beta \sqrt{x_i} + \mu_i $, for  i = 1, ..., n <br>
$ |\epsilon_i| = \alpha + \beta \frac{1}{x_i} + \mu_i $, for  i = 1, ..., n <br>


Step 4. Use t-test to check significance of all estimated $ \beta $'s <br>
if any of Betas is significant then we have heteroskedasticity



In [ ]:
# В лекциях/вики среди прочего есть модель с делением единицы на регрессор х 
# (x_div), что ведет к ошибкам если есть хотя бы один 0.
# НЕ СМОГ КОНВЕРТИРОВАТЬ ТИП ДАННЫХ В ТАБЛИЦЕ СТАТСМОДЕЛС(

"""
In this example, we load the "fair" dataset from the statsmodels library and fit a linear regression 
model to it using the OLS function. We then create a new DataFrame for the Glejser test and calculate 
the square root of the absolute residuals and the squared value of the "educ" variable. We fit another 
linear regression model to this DataFrame using the OLS function and print the summary of the results.

Note that in the Glejser test, we regress the square root of the absolute residuals on the independent 
variables, which in this case are the "educ" variable and its squared value. The null hypothesis is that 
the variance of the residuals is constant across all levels of the independent variables.
"""
import itertools

# Run the Glejser test

def glejser(results, model):
    output = pd.DataFrame(columns=['regressor_type', 'coef', 'std_err', 't_stat', 'P_value>|t|', '[0.025', '0.975]'])
    
    for j in range(len(model.exog_names)):
        
        # Create regressors and regressant for auxiliary regressions
        y = abs(results.resid)
        X_reg = model.exog[:, j]
        X_root = model.exog[:, j] ** 0.5
        X_sqr = model.exog[:, j] ** 2
        iterable = [X_reg, X_root, X_sqr]

        # Run the Glejser test and append it in dataframe (only Beta coef's are saved)

        for i in iterable:
            glejser_model = sm.OLS(y, sm.add_constant(i))
            glejser_results = glejser_model.fit()
            a = pd.DataFrame(glejser_results.summary().tables[1])
            a.rename(columns={a.columns[0]: 'regressor_type',
                          a.columns[1]: 'coef',
                          a.columns[2]: 'std_err',
                          a.columns[3]: 't_stat',
                          a.columns[4]: 'P_value>|t|',
                          a.columns[5]: '[0.025',
                          a.columns[6]: '0.975]',
                         }, inplace=True)
            a.drop(index=[0, 1], inplace=True)
            a['feature'] = model.exog_names[j]
            output = pd.concat([output, a], axis=0)


    
    # Create column with name of used auxiliary regressor

    num_cycle = itertools.cycle([1, 2, 3])
    output['regressor_type'] = [next(num_cycle) for num in range(len(output))]
    output['regressor_type'].replace([1, 2, 3], ['X_reg', 'X_root', 'X_sqr'], inplace=True)
    output.reset_index(drop=True, inplace=True)
    #output = output.infer_objects()
    #output = output.astype(dtype={'regressor_type': str, 'coef': float, 'std_err': float, 't_stat': float, 'P_value>|t|': float, '[0.025': float, '0.975]': float, 'feature': str})
    #output[['coef', 'std_err', 't_stat', 'P_value>|t|', '[0.025', '0.975]']] = output[['coef', 'std_err', 't_stat', 'P_value>|t|', '[0.025', '0.975]']].astype(float)
    
    return output


In [ ]:
#glejser_test = glejser(results, model)

In [ ]:
def settype(t):
    t.set_datatype(float)
    return t.data

In [ ]:
#glejser_test['t_stat'].apply(settype).astype(float)

### Breusch-Pagan test

H0: Homoskedasticity $ \sigma_i^2 = \sigma^2$,  $ \forall_1i = 1, ..., n $<br>
H1: Heteroskedasticity  $ \sigma_i^2 \sim f(\alpha_0 + \alpha_1 z_{1i} + ... + \alpha_r z_{ri}) $ <br>

Test of dependance of variance on other variables z


Step 1. Estimate regression <br>

Step 2. Predict residuals $ \epsilon_i $ and calculate RSS  <br>

Step 3. Find estimate for variance: <br>
$ \hat{\sigma_u^2} = \frac{RSS}{n} $ <br>

Step 4. Estimate regression: <br>
$ \epsilon_i^2 = \gamma_0 + \gamma_1 z_1 + ... + \gamma_r z_r $

Step 5. Calculate $ ESS_0 $ from regression in Step 4 <br>

Step 6. Calculate test statistic <br>

$ BP = \frac{ESS_0}{2\hat{\sigma^4}} \sim \chi_r^2 $ , where $ r $ is a number of variables z <br>


#### If we have homoskedasticity we can estimate regression easy <br>
$ Var(\hat{\beta}) = (X^\prime X)^{-1} X^\prime \Omega X (X^\prime X)^{-1} $, <br>
where $ \Omega $ is the covariance matrix.
In case of homoskedasticity $ \Omega $: <br>
$ \Omega = \begin{pmatrix} \sigma^2 & ... & 0\\ ... & ... & ...\\ 0 & ... & \sigma^2 \end{pmatrix}  $, <br>
hence equation simplifies to: <br>
$ Var(\hat{\beta}) = \sigma^2 (X^\prime X)^{-1}  $ <br>

#### But in case of heteroskedasticity $ \Omega $ isn't diagonal: <br>
hence equation will look like this: <br>
$ Var(\hat{\beta}) = (X^\prime X)^{-1} X^\prime (\sum_{i=1}^n \sigma_i^2 x_i x_i^\prime) X (X^\prime X)^{-1} $ <br>

##### 1. White's estimator (heteroskedasticity-consistent estimator, HCE): <br>
$ Var(\hat{\beta}) = (X^\prime X)^{-1} X^\prime (\sum_{i=1}^n \epsilon_i^2 x_i x_i^\prime) X (X^\prime X)^{-1} $ <br>

##### 2. Generalized Least Squares (GLS): <br>
##### 3. Feasible Generalized Least Squares (FGLS): <br>
##### 4. Weighted Least Squares: <br>


### Hypotheses:
1. Regions closer to the waterline add additional value to property cost - could be visualized with map
2. Price for property is affected by fluctuations in prices of oil
3. Property without metro station around cost less.
4. Commercial property cost more than residential
5. Increase in area positively influence propery price


### Need to add:
#### A couple more graphs to illustrate categoriacal features (barcharts, boxplots)

We have chosen OLS 

In [ ]:
# H0 = Model correctly spicified
# H1 = Model misspecified

ramsey_test = linear_reset(results)


In [ ]:
sm_data.head(3)

In [ ]:
data_try = sm_data
data_try['Price_Sq_m'] = data_try['Amount'] / data_try['Property_Size']

formula_try = 'Price_Sq_m ~ C(Transaction_Type) + C(Usage) + C(Area) + Property_Size + C(Nearest_Metro) + Oil_Price + C(Is_Free_Hold)'

model_try = smf.ols(formula=formula_try, data=data_try)
results_try = model_try.fit()

In [ ]:
results_try.summary()

In [ ]:
corr = data_try.select_dtypes("number").drop(columns=['const', 'Price_Sq_m']).corr()
sns.heatmap(corr);

#### 1A <br>
a - Берем нашу первую модель, можно добавить геопозицию <br>
б - Удобно протестировать (1. Что с площадью растет и цена 2. Что с приближением к воде растет цена) <br>
с - Можно трансформировать Y сказав, что будем прогнозировать Цена/Площадь <br>
или просто логарифмировать что-то, возвести в квадрат => сравнить модели F тестом <br>




#### 1Б <br>
д. Мне кажется можно описать модель с предыдущего шага. Описание мисспесификаций предположительных, __пока без тестов__. <br>
Можно трансформировать Y сказав, что будем прогнозировать Цена/Площадь <br>
или просто логарифмировать что-то, возвести в квадрат => сравнить модели F тестом <br>

е. Тесты на спецификацию: <br>
Перебор разных переменных, <br>
Еще трансформации, <br>
Тест Рамзей-РЕСЕТ, CHOW TEST (structural break) <br>

ф. Предфинальная модель - нормально специфицирована - предварительная интерпритация результатов <br>

#### 2 <br>
а - проверить мультиколлиниарность - практически сделал - VIF - после можно отбросить лишние параметры <br>
б - Гетероскедастичность - Тесты есть, можно попробовать избавиться <br>
с - Эндодженеити - Описание, что потенциально может быть эндодженос <br>
д - ФИНАЛЬНАЯ МОДЕЛЬ, ИНТЕРПРИТАЦИЯ, R^2, Подтвердились ли гипотезы? <br>